In [13]:
# import libraries
import numpy as np
import pandas as pd


In [20]:
# read data
data = pd.read_parquet('data/spx_ivs_2023-02.parquet')
data['date'] = pd.to_datetime(data["loctimestamp"], unit="ms").dt.date

In [ ]:
# Datei laden
df = pd.read_parquet("data/spx_ivs_2023-02.parquet")
df["date"] = pd.to_datetime(df["loctimestamp"], unit="ms").dt.date
df = df[df["daystomaturity"] == 30].copy()

# Gruppieren nach Datum
grouped = df.groupby("date")
results = []

for date, group in grouped:
    S = group["underlyingprice"].iloc[0]

    calls = group[(group["putcall"] == "C") & (group["strike"] >= S)].sort_values("strike")
    puts = group[(group["putcall"] == "P") & (group["strike"] < S)].sort_values("strike")

    Kc, Cc = calls["strike"].values, calls["implPrice"].values
    Kp, Pp = puts["strike"].values, puts["implPrice"].values

    # --- 2. Moment: Varianz ---
    Vc_integrand = (2 * (1 - np.log(Kc / S))) / (Kc ** 2) * Cc
    Vp_integrand = (2 * (1 + np.log(S / Kp))) / (Kp ** 2) * Pp
    V = np.trapz(Vc_integrand, Kc) + np.trapz(Vp_integrand, Kp)

    results.append({"date": date, "variance": V, "third_moment": W})

# Ergebnis als DataFrame
df_moments = pd.DataFrame(results)
print(df_moments.head())


         date  variance  third_moment
0  2023-02-01  0.002480      0.000036
1  2023-02-02  0.002853      0.000036
2  2023-02-03  0.002422      0.000036
3  2023-02-06  0.002840      0.000036
4  2023-02-07  0.002515      0.000036
